# Importing Libraries

In [1]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [2]:
df = pd.read_csv(r'C:\Users\user\Desktop\Dashboard\data\crypto.csv')

In [3]:
crypto = list(df['Crypto-USD'].unique())[1:]
crypto

['ETH-USD',
 'BNB-USD',
 'ADA-USD',
 'SOL-USD',
 'XRP-USD',
 'LUNA1-USD',
 'DOT-USD',
 'DOGE-USD',
 'AVAX-USD']

# Applying LSTM on BITCOIN

In [4]:
#Create a new data frame with only the Close Column
data = df[df['Crypto-USD'] == 'BTC-USD'].filter(["Close"])
#Convert te dataFrame to a numpy array
dataset = data.values
#Get the number of rows to train the model on
training_data_len = math.ceil(len(dataset)*0.8)
    
#scale the data
scaler = MinMaxScaler(feature_range = (0,1))
scaled_data = scaler.fit_transform(dataset)
    
#Create The training data set
#Create the scaled training data set

train_data = scaled_data[0:training_data_len , :]
#split the data into x_train and y_train data set
x_train = []
y_train = []

for i in range(60 , len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    
#Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)
    
#REshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
    
#Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1],1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

#compil the model
model.compile(optimizer = "adam" , loss = "mean_squared_error")

#Train the model
model.fit(x_train, y_train, batch_size=5, epochs=10)
#Create the testing data set
#Create a new array containing scaled values from index 1543 to 2003
test_data = scaled_data[training_data_len - 60 : , :]
#Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60 , len(test_data)):
    x_test.append(test_data[i-60:i,0])
    
#convert the data to a numpy array
x_test = np.array(x_test)
    
#reshape the data because LSTM expect 3 dimentional
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))

#get the models predicted price values
Predictions = model.predict(x_test)
Predictions = scaler.inverse_transform(Predictions)
#we want prediction to contain the same values as y_test
rmse = []
r_square  = []
    
rmse.append(np.sqrt(np.mean(Predictions - y_test)**2))
r_square.append(r2_score(y_test,Predictions))
    
#Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid["Predictions"] = Predictions

train['Crypto-USD'] = 'BTC-USD'
valid['Crypto-USD'] = 'BTC-USD'

Epoch 1/10
417/417 [==============================] - 15s 37ms/step - loss: 1.7371e-04
Epoch 2/10
417/417 [==============================] - 18s 43ms/step - loss: 9.3291e-05
Epoch 3/10
417/417 [==============================] - 17s 41ms/step - loss: 6.5690e-05
Epoch 4/10
417/417 [==============================] - 17s 40ms/step - loss: 5.7540e-05
Epoch 5/10
417/417 [==============================] - 16s 38ms/step - loss: 4.8116e-05
Epoch 6/10
417/417 [==============================] - 16s 39ms/step - loss: 4.8628e-05
Epoch 7/10
417/417 [==============================] - 18s 44ms/step - loss: 3.9647e-05
Epoch 8/10
417/417 [==============================] - 16s 40ms/step - loss: 4.2983e-05
Epoch 9/10
417/417 [==============================] - 19s 45ms/step - loss: 3.4221e-05
Epoch 10/10
417/417 [==============================] - 17s 40ms/step - loss: 3.3822e-05


<ipython-input-4-18209b2d71bb>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid["Predictions"] = Predictions
<ipython-input-4-18209b2d71bb>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Crypto-USD'] = 'BTC-USD'
<ipython-input-4-18209b2d71bb>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [5]:
valid

,Close,Predictions,Crypto-USD
2143,11111.213867,10944.210938,BTC-USD
2144,11323.466797,10951.977539,BTC-USD
2145,11759.592773,11141.704102,BTC-USD
2146,11053.614258,11554.143555,BTC-USD
2147,11246.348633,10952.218750,BTC-USD
...,...,...,...
2673,42735.855469,38948.812500,BTC-USD
2674,43949.101562,40013.957031,BTC-USD
2675,42591.570312,41226.343750,BTC-USD
2676,43099.699219,39307.988281,BTC-USD


# Applying LSTM on other crytos using a loop

In [6]:
for coin in crypto:
    #Create a new data frame with only the Close Column
    data = df[df['Crypto-USD'] == coin].filter(["Close"])
    #Convert te dataFrame to a numpy array
    dataset = data.values
    #Get the number of rows to train the model on
    training_data_len = math.ceil(len(dataset)*0.8)
    
    #scale the data
    scaler = MinMaxScaler(feature_range = (0,1))
    scaled_data = scaler.fit_transform(dataset)

    #Create The training data set
    #Create the scaled training data set

    train_data = scaled_data[0:training_data_len , :]
    #split the data into x_train and y_train data set
    x_train = []
    y_train = []

    for i in range(60 , len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])
    
    #Convert the x_train and y_train to numpy arrays
    x_train, y_train = np.array(x_train), np.array(y_train)
    
    #REshape the data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
    
    #Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1],1)))
    model.add(LSTM(50, return_sequences = False))
    model.add(Dense(25))
    model.add(Dense(1))
    
    #compil the model
    model.compile(optimizer = "adam" , loss = "mean_squared_error")
    
    #Train the model
    model.fit(x_train, y_train, batch_size=5, epochs=10)
    
    #Create the testing data set
    #Create a new array containing scaled values from index 1543 to 2003
    test_data = scaled_data[training_data_len - 60 : , :]
    #Create the data sets x_test and y_test
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60 , len(test_data)):
        x_test.append(test_data[i-60:i,0])
    
    #convert the data to a numpy array
    x_test = np.array(x_test)
    
    #reshape the data because LSTM expect 3 dimentional
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))
    
    #get the models predicted price values
    Predictions = model.predict(x_test)
    Predictions = scaler.inverse_transform(Predictions)
    #we want prediction to contain the same values as y_test
    
    
    
    rmse.append(np.sqrt(np.mean(Predictions - y_test)**2))
    r_square.append(r2_score(y_test,Predictions))
    
    train1 = data[:training_data_len]
    valid1 = data[training_data_len:]
    valid1["Predictions"] = Predictions
    
    train1['Crypto-USD'] = coin
    valid1['Crypto-USD'] = coin
    
    train = pd.concat([train,train1])
    valid = pd.concat([valid,valid1])
    

Epoch 1/10
233/233 [==============================] - 10s 42ms/step - loss: 7.6517e-04
Epoch 2/10
233/233 [==============================] - 8s 36ms/step - loss: 3.1219e-04
Epoch 3/10
233/233 [==============================] - 12s 49ms/step - loss: 2.0543e-04
Epoch 4/10
233/233 [==============================] - 8s 36ms/step - loss: 1.8743e-04
Epoch 5/10
233/233 [==============================] - 8s 33ms/step - loss: 1.6000e-04
Epoch 6/10
233/233 [==============================] - 10s 44ms/step - loss: 1.5469e-04
Epoch 7/10
233/233 [==============================] - 9s 40ms/step - loss: 1.4803e-04
Epoch 8/10
233/233 [==============================] - 11s 45ms/step - loss: 1.0843e-04
Epoch 9/10
233/233 [==============================] - 8s 34ms/step - loss: 1.0001e-04
Epoch 10/10
233/233 [==============================] - 9s 39ms/step - loss: 1.0655e-04


<ipython-input-6-a6ee6784117e>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1["Predictions"] = Predictions
<ipython-input-6-a6ee6784117e>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train1['Crypto-USD'] = coin
<ipython-input-6-a6ee6784117e>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

Epoch 1/10
233/233 [==============================] - 9s 39ms/step - loss: 4.6106e-04
Epoch 2/10
233/233 [==============================] - 8s 35ms/step - loss: 3.7467e-04
Epoch 3/10
233/233 [==============================] - 9s 39ms/step - loss: 2.2680e-04
Epoch 4/10
233/233 [==============================] - 8s 35ms/step - loss: 1.9797e-04
Epoch 5/10
233/233 [==============================] - 9s 37ms/step - loss: 2.0080e-04
Epoch 6/10
233/233 [==============================] - 8s 35ms/step - loss: 2.3189e-04
Epoch 7/10
233/233 [==============================] - 9s 41ms/step - loss: 2.3219e-04: 0s - loss: 2.3546e
Epoch 8/10
233/233 [==============================] - 10s 41ms/step - loss: 1.7417e-04
Epoch 9/10
233/233 [==============================] - 10s 42ms/step - loss: 1.4940e-04
Epoch 10/10
233/233 [==============================] - 9s 38ms/step - loss: 1.4755e-04: 1s - l


<ipython-input-6-a6ee6784117e>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1["Predictions"] = Predictions
<ipython-input-6-a6ee6784117e>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train1['Crypto-USD'] = coin
<ipython-input-6-a6ee6784117e>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

Epoch 1/10
233/233 [==============================] - 9s 39ms/step - loss: 7.5321e-04
Epoch 2/10
233/233 [==============================] - 8s 36ms/step - loss: 3.0036e-04
Epoch 3/10
233/233 [==============================] - 9s 41ms/step - loss: 2.2929e-04
Epoch 4/10
233/233 [==============================] - 9s 37ms/step - loss: 2.5693e-04
Epoch 5/10
233/233 [==============================] - 10s 41ms/step - loss: 1.7755e-04
Epoch 6/10
233/233 [==============================] - 9s 40ms/step - loss: 1.7545e-04
Epoch 7/10
233/233 [==============================] - 9s 37ms/step - loss: 1.4913e-04
Epoch 8/10
233/233 [==============================] - 8s 36ms/step - loss: 1.4360e-04
Epoch 9/10
233/233 [==============================] - 9s 38ms/step - loss: 1.3545e-04
Epoch 10/10
233/233 [==============================] - 8s 35ms/step - loss: 1.5368e-04


...

<ipython-input-6-a6ee6784117e>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1["Predictions"] = Predictions
<ipython-input-6-a6ee6784117e>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train1['Crypto-USD'] = coin
<ipython-input-6-a6ee6784117e>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

Epoch 1/10
233/233 [==============================] - 9s 40ms/step - loss: 7.3571e-05
Epoch 2/10
233/233 [==============================] - 11s 49ms/step - loss: 3.1493e-05
Epoch 3/10
233/233 [==============================] - 10s 42ms/step - loss: 1.9737e-05
Epoch 4/10
233/233 [==============================] - 11s 46ms/step - loss: 1.8185e-05
Epoch 5/10
233/233 [==============================] - 10s 45ms/step - loss: 1.8156e-05
Epoch 6/10
233/233 [==============================] - 11s 49ms/step - loss: 1.7152e-05
Epoch 7/10
233/233 [==============================] - 11s 49ms/step - loss: 1.6680e-05
Epoch 8/10
233/233 [==============================] - 11s 45ms/step - loss: 1.6177e-05
Epoch 9/10
233/233 [==============================] - 10s 44ms/step - loss: 1.4040e-05
Epoch 10/10
233/233 [==============================] - 10s 44ms/step - loss: 1.3448e-05


<ipython-input-6-a6ee6784117e>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1["Predictions"] = Predictions
<ipython-input-6-a6ee6784117e>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train1['Crypto-USD'] = coin
<ipython-input-6-a6ee6784117e>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

Epoch 1/10
66/66 [==============================] - 2s 33ms/step - loss: 0.0072
Epoch 2/10
66/66 [==============================] - 3s 38ms/step - loss: 0.0024
Epoch 3/10
66/66 [==============================] - 3s 40ms/step - loss: 0.0019
Epoch 4/10
66/66 [==============================] - 3s 40ms/step - loss: 0.0017
Epoch 5/10
66/66 [==============================] - 3s 45ms/step - loss: 0.0016
Epoch 6/10
66/66 [==============================] - 3s 47ms/step - loss: 0.0014
Epoch 7/10
66/66 [==============================] - 2s 32ms/step - loss: 0.0016
Epoch 8/10
66/66 [==============================] - 3s 40ms/step - loss: 0.0011
Epoch 9/10
66/66 [==============================] - 3s 45ms/step - loss: 0.0011
Epoch 10/10
66/66 [==============================] - 3s 42ms/step - loss: 0.0012


<ipython-input-6-a6ee6784117e>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1["Predictions"] = Predictions
<ipython-input-6-a6ee6784117e>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train1['Crypto-USD'] = coin
<ipython-input-6-a6ee6784117e>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [7]:
train

,Close,Crypto-USD
0,457.334015,BTC-USD
1,424.440002,BTC-USD
2,394.795990,BTC-USD
3,408.903992,BTC-USD
4,398.821014,BTC-USD
...,...,...
12762,61.488354,AVAX-USD
12763,60.520226,AVAX-USD
12764,60.566257,AVAX-USD
12765,56.593575,AVAX-USD


In [8]:
r_square

[0.9569168328112613,
 0.9559731378263865,
 0.7811383367565484,
 0.9212215809628679,
 -11.512322254194611,
 0.766700747023203,
 0.8962176884634908,
 0.8971399750031659,
 0.870331523881273,
 0.7281715413792598]

In [9]:
rmse

[2670.4870728679907,
 44.91785748591189,
 38.76560173660028,
 0.06352449362395239,
 102.56604559846626,
 0.08561395709631874,
 5.043911520288794,
 0.05097955760389261,
 0.017096347789295356,
 8.004742900530497]

# Creating DataFrame of the performance of LSTM in each Crypto

In [10]:
dictionary = {'Crypto-USD':list(df['Crypto-USD'].unique()),'R_square': r_square,"RMSE":rmse}
performance = pd.DataFrame(dictionary)
performance

,Crypto-USD,R_square,RMSE
0,BTC-USD,0.956917,2670.487073
1,ETH-USD,0.955973,44.917857
2,BNB-USD,0.781138,38.765602
3,ADA-USD,0.921222,0.063524
4,SOL-USD,-11.512322,102.566046
5,XRP-USD,0.766701,0.085614
6,LUNA1-USD,0.896218,5.043912
7,DOT-USD,0.897140,0.050980
8,DOGE-USD,0.870332,0.017096
9,AVAX-USD,0.728172,8.004743


In [11]:
valid

,Close,Predictions,Crypto-USD
2143,11111.213867,10944.210938,BTC-USD
2144,11323.466797,10951.977539,BTC-USD
2145,11759.592773,11141.704102,BTC-USD
2146,11053.614258,11554.143555,BTC-USD
2147,11246.348633,10952.218750,BTC-USD
...,...,...,...
12858,89.501816,77.348267,AVAX-USD
12859,95.823509,78.119026,AVAX-USD
12860,89.407883,81.647720,AVAX-USD
12861,90.340530,83.817398,AVAX-USD


# Saving to csv

In [13]:
performance.to_csv(r'C:\Users\user\Desktop\Dashboard\data\performance.csv')
train.to_csv(r'C:\Users\user\Desktop\Dashboard\data\train.csv')
valid.to_csv(r'C:\Users\user\Desktop\Dashboard\data\valid.csv')

In [16]:

import plotly.express as px
import plotly.graph_objects as go
valid1 = valid[valid['Crypto-USD'] == 'DOGE-USD']
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(valid1['Close']))), y=list(valid1['Close']), name='Closing Price'))
fig.add_trace(go.Scatter(y=list(valid1['Predictions']), name='Prediction'))
fig.update_layout(legend_title_text='Legend')

In [ ]:
len(list(valid1['Close']))